In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Translation Evaluation without reference.

## Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform[evaluation]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00


## Authenticate to access GCP

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

    from google.colab import drive
    drive.mount('/content/drive')

!gcloud config set project ai-hangsik

Mounted at /content/drive
Updated property [core/project].


## Initialize Vertex AI and Gemini model

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL = "gemini-1.5-pro-002"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

import pandas as pd
pd.options.display.max_colwidth = 300

vertexai.init(project=PROJECT_ID, location=LOCATION)
# model = GenerativeModel(MODEL)

## Evaluation with MetricX
### Reference free (Quality Estimation, QE)

* Reference 가 없는, 새로운 컨텐츠에 대한 번역과 이를 MetricX를 이용해 reference free (Quality Estimation, QE) 모드로 평가하는 예시입니다.

In [4]:
cd "/content/drive/MyDrive/Colab Notebooks/projects/TVPlus_Translation/notebook/translation"

/content/drive/MyDrive/Colab Notebooks/projects/TVPlus_Translation/notebook/translation


In [5]:
ls -al

total 202
-rw------- 1 root root  4642 Jan  6 04:27  adaptive_proverb.tsv
-rw------- 1 root root 16669 Jan  6 15:37  adaptive_translation.ipynb
-rw------- 1 root root 15733 Dec 11 00:56 ''$'\b''audio_transcription.ipynb'
-rw------- 1 root root 15249 Jan  6 14:41  check_language_support.ipynb
-rw------- 1 root root  4742 Jan  6 14:41  detect_language.ipynb
-rw------- 1 root root   144 Jan  6 01:42  glossary_ko.csv
-rw------- 1 root root  7249 Jan  6 14:41  text_translation_v3.ipynb
-rw------- 1 root root  5105 Jan  6 15:21  transcript_translation_br.csv
-rw------- 1 root root  5134 Jan  6 15:13  transcript_translation.csv
-rw------- 1 root root 13294 Jan  6 14:41  translate_glossary.ipynb
-rw------- 1 root root 38807 Jan  6 15:37  video_transcription_2.0.ipynb
-rw------- 1 root root 77285 Jan  6 15:23  video_transcription_translation.ipynb


In [6]:
import pandas as pd

file_name = "transcript_translation_br.csv"
translation = pd.read_csv(file_name)
translation.head()

translation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   44 non-null     int64 
 1   speaker      44 non-null     object
 2   transcript   44 non-null     object
 3   translation  44 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.5+ KB


In [7]:
dataset = translation[['transcript', 'translation']].rename(columns={'transcript': 'source', 'translation': 'response'})
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   source    44 non-null     object
 1   response  44 non-null     object
dtypes: object(2)
memory usage: 832.0+ bytes


In [8]:
from vertexai.generative_models import Part, Tool, grounding
from vertexai.evaluation import EvalTask, PointwiseMetric, PairwiseMetric
import vertexai.evaluation.metrics.pointwise_metric as pointwise_metric

eval_task = EvalTask(dataset=dataset,
                    metrics=[pointwise_metric.MetricX(version="METRICX_24_SRC")],
                    experiment="pointwise-eval")

result_pointwise = eval_task.evaluate()
result_pointwise.metrics_table


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/pointwise-eval-0dc0caa0-4787-4b8f-ac19-8b4d6537969d to Experiment: pointwise-eval


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 44 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 44/44 [00:47<00:00,  1.08s/it]
INFO:vertexai.evaluation._evaluation:All 44 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:47.4795471220001 seconds


,source,response,metricx/score
0,옥황상제님의 아드님께서 오신다! 예를 갖춰라!,O filho do Imperador de Jade está vindo! Preparem-se para mostrar respeito!,2.720117
1,지상의 왕은 황금 1만냥을 함경도 기근 지역에 보냈느냐?,O rei terreno enviou os 10.000 nyang de ouro para a área de fome em Hamgyong?,4.632004
2,"아, 예. 그제 제 꿈에 나타나 하명하신대로 한 치 틀림없이 그리 하였습니다.","Ah, sim. Exatamente como você ordenou em meu sonho outro dia, eu o fiz sem nenhum erro.",1.770809
3,"하늘에서 그대의 덕을 높이사 그대가 하늘로 돌아올 때 7배, 70배, 700배로 갚아줄 것이다.","O céu aprecia sua virtude e a recompensará sete vezes, setenta vezes, setecentas vezes quando você retornar ao céu.",0.694741
4,왕가의 보물을 보자시기에 그건 역시 준비했습니다.,"Como você queria ver os tesouros da família real, eu também os preparei.",3.996182
5,지상의 왕이 보기보다 아주 똘똘하구나.,O rei terreno é mais inteligente do que parece.,2.385403
6,예.,Sim.,2.627782
7,헌데 궁녀들은 어디에?,Mas onde estão as damas da corte?,0.995318
8,궁녀들은 어찌하여 그러시는지.,Por que você pergunta sobre as damas da corte?,9.086471
9,너는 연불한다고 궁녀들이 술 좀 따라주면 좋겠는데.,Eu gostaria que as damas da corte me servissem um pouco de bebida enquanto eu oro.,4.693907
